In [1]:
!pip install pinecone-client

  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pinecone-plugin-inference-3.1.0:
      Successfully uninstalled pinecone-plugin-inference-3.1.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [10]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pinecone import Pinecone
pc = Pinecone(api_key="pcsk_3CYnJi_TZbGr8CeCcVxAsz4Li7J5n5hNBRqM7PA7k6xGKx7ftNXUYMYUJLJcb3PZrTneH4") # Replace with your actual API key


In [12]:
def retrieve_topics_from_namespaces(index, namespaces):
    """
    Retrieve topics from multiple namespaces and store them in separate lists.

    :param index: Pinecone Index object.
    :param namespaces: List of namespaces to query.
    :return: A tuple containing two lists: one for topics in the first namespace and one for topics in the second namespace.
    """
    topics_by_namespace = {}

    for namespace in namespaces:
        topics = set()  # Using set to avoid duplicate topics
        try:
            # Query with an empty vector to fetch all items in the namespace
            response = index.query(vector=[0] * 768, namespace=namespace, top_k=1000, include_metadata=True)
            # Extract topics from the metadata
            for match in response["matches"]:
                metadata = match.get("metadata", {})
                if "topic" in metadata:
                    topics.add(metadata["topic"])
        except Exception as e:
            print(f"Error retrieving metadata from namespace '{namespace}': {e}")

        # Store the topics for each namespace
        topics_by_namespace[namespace] = list(topics)  # Convert set to list

    return topics_by_namespace

# Example usage:
index_name = "mcq-index"
index = pc.Index(index_name)

# Specify the namespaces
namespaces = ["computer_organization", "operating_system"]

# Retrieve topics from both namespaces
topics_by_namespace = retrieve_topics_from_namespaces(index, namespaces)
print(topics_by_namespace)
# Print topics for each namespace
for namespace, topics in topics_by_namespace.items():
    print(f"Topics in namespace '{namespace}':")
    for topic in topics:
        print(f"- {topic}")


{'computer_organization': ['Digital Logic Number Representation', 'Computer Organization Architecture', 'Number Representation', 'Microprocessor'], 'operating_system': ['Memory Management', 'Input Output Systems', 'Cpu Scheduling', 'Unix', 'Dead Lock', 'Process Management']}
Topics in namespace 'computer_organization':
- Digital Logic Number Representation
- Computer Organization Architecture
- Number Representation
- Microprocessor
Topics in namespace 'operating_system':
- Memory Management
- Input Output Systems
- Cpu Scheduling
- Unix
- Dead Lock
- Process Management


In [23]:
# prompt: above is code is only giving result from one namespace i need best match of 10 

from pinecone import Pinecone
import pinecone
from sentence_transformers import SentenceTransformer

def retrieve_topics_from_namespaces(index, namespaces):
    """
    Retrieve topics from multiple namespaces and store them in separate lists.

    :param index: Pinecone Index object.
    :param namespaces: List of namespaces to query.
    :return: A tuple containing two lists: one for topics in the first namespace and one for topics in the second namespace.
    """
    topics_by_namespace = {}

    for namespace in namespaces:
        topics = set()  # Using set to avoid duplicate topics
        try:
            # Query with an empty vector to fetch all items in the namespace
            response = index.query(vector=[0] * 768, namespace=namespace, top_k=1000, include_metadata=True)
            # Extract topics from the metadata
            for match in response["matches"]:
                metadata = match.get("metadata", {})
                if "topic" in metadata:
                    topics.add(metadata["topic"])
        except Exception as e:
            print(f"Error retrieving metadata from namespace '{namespace}': {e}")

        # Store the topics for each namespace
        topics_by_namespace[namespace] = list(topics)  # Convert set to list

    return topics_by_namespace

def search_mcqs_by_query(index, query, namespaces, top_k=10):
    """
    Searches for MCQs across multiple namespaces and returns the best matches.
    """
    model = SentenceTransformer('all-mpnet-base-v2')
    query_embedding = model.encode(query)
    all_results = []

    for namespace in namespaces:
        try:
            response = index.query(vector=query_embedding.tolist(), namespace=namespace, top_k=top_k, include_metadata=True)
            for match in response["matches"]:
                all_results.append({
                    "text": match["id"],
                    "metadata": match["metadata"],
                    "score": match["score"],
                    "namespace": namespace
                })
        except Exception as e:
            print(f"Error searching namespace '{namespace}': {e}")

    # Sort all results by score in descending order
    all_results.sort(key=lambda x: x["score"], reverse=True)
    return all_results[:top_k]  # Return top_k best matches across all namespaces


# Example usage
pc = Pinecone(api_key="pcsk_3CYnJi_TZbGr8CeCcVxAsz4Li7J5n5hNBRqM7PA7k6xGKx7ftNXUYMYUJLJcb3PZrTneH4", environment="us-west1-gcp")
index_name = "mcq-index"
index = pc.Index(index_name)

namespaces = ["computer_organization", "operating_system"]

search_query = input("Enter your search query: ")
mcq_results = search_mcqs_by_query(index, search_query, namespaces)


if mcq_results:
    print(f"Top {len(mcq_results)} MCQs for '{search_query}' across namespaces:")
    for result in mcq_results:
        print(f"Text: {result['text']}")
        print(f"Metadata: {result['metadata']}")
        print(f"Score: {result['score']}")
        print(f"Namespace: {result['namespace']}")
        print("---")
else:
    print(f"No MCQs found for '{search_query}'.")

Top 10 MCQs for 'number representation' across namespaces:
Text: Number Representation-Question25
Metadata: {'correct_option': 'C1CE0000', 'options': ['61CD0000', 'C1640000', 'C1CD0000', 'C1CE0000'], 'question_img_link': '', 'question_no': 'Question25', 'question_text': 'The value of a float type variable is represented using the single-precision 32-bit floating point format IEEE-754 standard that uses 1 bit for sign, 8 bits for biased exponent and 23 bits for mantissa. A float type variable X is assigned the decimal value of −25.75. The representation of X in hexadecimal notation is', 'topic': 'Number Representation'}
Score: 0.406145632
Namespace: computer_organization
---
Text: Number Representation-Question7
Metadata: {'correct_option': 'C1640000H', 'options': ['C1640000H', '416C0000H', '41640000H', 'C16C0000H'], 'question_img_link': '', 'question_no': 'Question7', 'question_text': 'The value of a float type variable is represented using the single-precision 32-bit floating point fo